In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm

In [ ]:
df_all = []
file_lst = os.listdir('NMXLNT')
for file_name in file_lst:
    df = pd.read_csv(f"NMXLNT/{file_name}")
    df = df.drop_duplicates().reset_index().drop("index", axis = 1) # after some inspection, realize there's some duplicates
    df_all.append(df)
    print(file_name)
    print(df.columns)

In [ ]:
df_all[0]

In [ ]:
df_all[0].info()

In [ ]:
df_all[0]["datetime"] = pd.to_datetime(df_all[0]["datetime"])

In [ ]:
sns.lineplot(df_all[0], x = "datetime", y = "cod")
# seems to have some skip in time?

In [ ]:
# check if what time do we have
for inx, df in enumerate(df_all):
    df["datetime"] = pd.to_datetime(df["datetime"])
    df = df.sort_values(by = "datetime", ascending = True).reset_index().drop("index", axis = 1)
    print(file_lst[inx])
    print(df["datetime"].min(), df["datetime"].max()) # we have 2024
    print(df["datetime"].dt.date.nunique()) # missing a lot of days

In [ ]:
# try to plot cod against time in all data
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    x, y = inx // 3, inx % 3
    sns.scatterplot(df, x = "datetime", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# missing many dates, might not use lagged date features, lagged in date feature might be preferable

In [ ]:
# try to plot cod against time in all data
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    x, y = inx // 3, inx % 3
    sns.histplot(df["cod"], ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# missing many dates, might not use lagged date features, lagged in date feature might be preferable

In [ ]:
# check if how many time do we get the datat in a day
# assume we collect data every 5 mins, from 12AM to 11:55 PM => we have 288 datapoints per day
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df["date"] = df["datetime"].dt.date
    num_cod_per_date = df.groupby("date")["datetime"].count()
    x, y = inx // 3, inx % 3
    sns.histplot(num_cod_per_date, ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# not always have that, might want to check if the time is equally spaced at 5 min

In [ ]:
# check if how many time do we get the data in a day
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df["date"] = df["datetime"].dt.date
    num_cod_per_date = df.groupby("date")["datetime"].apply(lambda x: None if x.count() == 1 else (x.max() - x.min()) / (pd.Timedelta(minutes=1) * (x.count() - 1)))
    x, y = inx // 3, inx % 3
    sns.histplot(num_cod_per_date, ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# data is equally spaced around 5 min => can use last few timestamps in around 5 mins

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    temp_df["prev_datetime"] = temp_df["datetime"].shift(1)
    temp_df["prev_cod"] = temp_df["cod"].shift(1)
    # filter those in which previous timestap is less than 10 min
    temp_df = temp_df[(temp_df["datetime"] - temp_df["prev_datetime"]) / pd.Timedelta(minutes=1) <= 10].reset_index().drop("index", axis = 1)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "prev_cod", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    temp_df["prev_datetime"] = temp_df["datetime"].shift(2)
    temp_df["prev_cod_2"] = temp_df["cod"].shift(2)
    # filter those in which previous timestap is less than 10 min
    temp_df = temp_df[(temp_df["datetime"] - temp_df["prev_datetime"]) / pd.Timedelta(minutes=1) <= 20].reset_index().drop("index", axis = 1)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "prev_cod_2", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    temp_df["prev_datetime"] = temp_df["datetime"].shift(3)
    temp_df["prev_cod_3"] = temp_df["cod"].shift(3)
    # filter those in which previous timestap is less than 10 min
    temp_df = temp_df[(temp_df["datetime"] - temp_df["prev_datetime"]) / pd.Timedelta(minutes=1) <= 30].reset_index().drop("index", axis = 1)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "prev_cod_3", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
# fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
# for inx, df in enumerate(df_all):
#     df = df.reset_index().drop("index", axis = 1)
#     last_row_with_time_within_1hour = []
#     curr_row = 0
#     for i in tqdm(range(df.shape[0])):
#         while curr_row < i and (df.loc[i, "datetime"] - df.loc[curr_row, "datetime"]) / pd.Timedelta(minutes=1) > 60:
#             curr_row += 1
#         # when we stop is when we do not see a fit row, try to check if that fit row actually fit (curr_row - 1 would be our answer)
#         if curr_row < i:
#             last_row_with_time_within_1hour.append(curr_row)
#         else:
#             # we stop because we have no answer
#             last_row_with_time_within_1hour.append(None)
#     df["last_row_with_time_within_1hour"] = last_row_with_time_within_1hour
#     df2 = deepcopy(df.reset_index())
#     df = df.merge(df2, how = "inner", left_on = "last_row_with_time_within_1hour", right_on = "index", suffixes=("", "_prev"))
#     x, y = inx // 3, inx % 3
#     sns.scatterplot(temp_df, x = "cod_prev", y = "cod", ax = ax[x][y])
#     ax[x][y].set_title(file_lst[inx])          
# plt.show()

In [ ]:
# We will try to get data of around  4-8 hours before current time and see if its has correlation with current data
# before around k hours: from k - before k + 1 hours
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    print(file_lst[inx])
    df = df.reset_index().drop("index", axis = 1)
    last_row_with_time_within = {
        j: [] for j in range(4, 9) # store between 4 - 8 hours since we are trying to predict the next 4-8 hours
    }
    curr_row = {
        j: 0 for j in range(4, 9)
    }
    for i in tqdm(range(df.shape[0])):
        for j in range(4, 9):
            # we first try to find until we find first instance that is STRICTLY LESS THAN k hours before current time
            # then the previous instance is the instance that might before around 4 hours before current time
            while curr_row[j] < i and (df.loc[i, "datetime"] - df.loc[curr_row[j], "datetime"]) / pd.Timedelta(minutes=1) >= 60 * j:
                curr_row[j] += 1
            # when we stop is when we do not see a fit row, try to check if that fit row actually fit 
            # a fit would be from k - < k+1 hours before (curr_row - 1 would be our answer)
            if curr_row[j] <= i and curr_row[j] >= 1 and \
               (df.loc[i, "datetime"] - df.loc[curr_row[j] - 1, "datetime"]) / pd.Timedelta(minutes=1) >= 60 * j and \
               (df.loc[i, "datetime"] - df.loc[curr_row[j] - 1, "datetime"]) / pd.Timedelta(minutes=1) < 60 * (j+1):
                last_row_with_time_within[j].append(curr_row[j] - 1)
            else:
                # we stop because we have no answer
                last_row_with_time_within[j].append(None)
    for j in range(4, 9):
        df[f"last_row_with_time_within_{j}hour"] = last_row_with_time_within[j]
    # need to separete these two steps since after we inner join, we will lose some rows
    for j in range(4, 9):
        df2 = deepcopy(df_all[inx].reset_index())
        df = df.merge(df2, how = "inner", left_on = f"last_row_with_time_within_{j}hour", right_on = "index", suffixes=("", f"_prev_{j}"))
        print(np.mean(((df["datetime"] - df[f"datetime_prev_{j}"]) / pd.Timedelta(minutes=1)) >= 60*j))
    df_all[inx] = deepcopy(df)
    x, y = inx // 3, inx % 3
    sns.scatterplot(df_all[inx], x = "cod_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])          
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    x, y = inx // 3, inx % 3
    sns.scatterplot(df_all[inx], x = "cod_prev_5", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])          
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    x, y = inx // 3, inx % 3
    sns.scatterplot(df_all[inx], x = "cod_prev_6", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])          
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    x, y = inx // 3, inx % 3
    sns.scatterplot(df_all[inx], x = "cod_prev_7", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])          
plt.show()

Flow In

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    if "flow_in_prev_4" not in temp_df.columns:
        temp_df["flow_in_prev_4"] = temp_df["flow_in1_prev_4"].fillna(0) + temp_df["flow_in2_prev_4"].fillna(0)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "flow_in_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# very noisy correlation, cau nga and tu son does not have data

Flow out

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "flow_out1_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# the trends of flow_out vs cod seems to have 2 ways: the U shape or the box as in caunga

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "flow_out2_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "flow_out3_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

Temperature

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "temp_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    temp_df["temp_above_26_prev_4"] = temp_df["temp_prev_4"] > 26
    # plot
    x, y = inx // 3, inx % 3
    sns.boxplot(temp_df, x = "temp_above_26_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "temp_prev_5", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "temp_prev_6", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "temp_prev_7", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

PH

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "ph_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# seems to be higher the closer it is to 7

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "ph_prev_5", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# seems to be higher the closer it is to 7

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "ph_prev_6", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# seems to be higher the closer it is to 7

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "ph_prev_7", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# seems to be higher the closer it is to 7

TSS

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "tss_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "tss_prev_5", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "tss_prev_6", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "tss_prev_7", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

Do

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "do_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# almost missing all in all factories, not really have any correlation

No3

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "no3_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# seems to not really have effect?

NH4

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "nh4_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# might have a trend but not really work for all factory

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "nh4_prev_5", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# might have a trend but not really work for all factory

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "nh4_prev_6", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# might have a trend but not really work for all factory

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "nh4_prev_7", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# might have a trend but not really work for all factory

PO4

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "po4_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# only 2 have it, slight trend

total_n

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "total_n_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

total_p

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "total_p_prev_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()
# seems to have a trend

Try to look at diff

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df_all[inx]["cod_diff_4"] = df_all[inx]["cod"] - df_all[inx]["cod_prev_4"]
    temp_df = deepcopy(df_all[inx])
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "ph_prev_4", y = "cod_diff_4", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    #df_all[inx]["cod_diff_4"] = df_all[inx]["cod"] - df_all[inx]["cod_prev_4"]
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "temp_prev_4", y = "cod_diff_4", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    #df_all[inx]["cod_diff_4"] = df_all[inx]["cod"] - df_all[inx]["cod_prev_4"]
    temp_df = deepcopy(df)
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "tss_prev_4", y = "cod_diff_4", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df_all[inx]["ph_diff_4"] = df_all[inx]["ph"] - df_all[inx]["ph_prev_4"]
    temp_df = deepcopy(df_all[inx])
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "ph_diff_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df_all[inx]["temp_diff_4"] = df_all[inx]["temp"] - df_all[inx]["temp_prev_4"]
    temp_df = deepcopy(df_all[inx])
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "temp_diff_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df_all[inx]["tss_diff_4"] = df_all[inx]["tss"] - df_all[inx]["tss_prev_4"]
    temp_df = deepcopy(df_all[inx])
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "tss_diff_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (20, 20))
for inx, df in enumerate(df_all):
    df_all[inx]["nh4_diff_4"] = df_all[inx]["nh4"] - df_all[inx]["nh4_prev_4"]
    temp_df = deepcopy(df_all[inx])
    # plot
    x, y = inx // 3, inx % 3
    sns.scatterplot(temp_df, x = "nh4_diff_4", y = "cod", ax = ax[x][y])
    ax[x][y].set_title(file_lst[inx])
plt.show()